In [7]:
import torch
import torch.nn as nn
import torchvision
#import torchvision.transforms as transforms
#from resnet import ResNet18
import torchvision.models as models

import numpy
import re
from typing import Dict,Iterable, Callable

#num_classes         = 10
model = models.resnet18(pretrained=True)
#model   = ResNet18(num_classes=num_classes, dropout=0.0)
trained_model_path  = "C:/Users/mmallikarjuna6/Downloads/resnet18_weights.pth"
model.load_state_dict(torch.load(trained_model_path))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
features = {}
def get_features(name):
    def hook(model, input, output):
        features[name] = output.cpu().detach()
    return hook

raw_layers = []
for layer in model.named_modules():
    raw_layers.append(layer[0])

print("\n\nRaw Layers:\n",raw_layers)

leaf_layers = []
for i in range(1, len(raw_layers)-1):
    curr_layer = raw_layers[i]
    next_layer = raw_layers[i+1]
    if(next_layer[:len(curr_layer)+1] != curr_layer + "."):
        leaf_layers.append(curr_layer)

leaf_layers.append(next_layer)
print("\n\nLeaf Layers:\n", leaf_layers)



Raw Layers:
 ['', 'conv1', 'bn1', 'relu', 'maxpool', 'layer1', 'layer1.0', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.1', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer2', 'layer2.0', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.downsample', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.1', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu', 'layer2.1.conv2', 'layer2.1.bn2', 'layer3', 'layer3.0', 'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu', 'layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.downsample', 'layer3.0.downsample.0', 'layer3.0.downsample.1', 'layer3.1', 'layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.relu', 'layer3.1.conv2', 'layer3.1.bn2', 'layer4', 'layer4.0', 'layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.relu', 'layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.downsample', 'layer4.0.downsample.0', 'layer4.0.downsample.1', 'layer4

In [9]:
layers = []
for i in range(len(leaf_layers)):
    layers.append(re.sub(r"\.(\d)",r"[\1]",leaf_layers[i]))
    
for i in range(len(layers)):
    layer = layers[i]
    layer_hook = "model." + layer + ".register_forward_hook(get_features('" + layer + "'))"
    exec(layer_hook)

print("\n\nLayers:\n", layers)



Layers:
 ['conv1', 'bn1', 'relu', 'maxpool', 'layer1[0].conv1', 'layer1[0].bn1', 'layer1[0].relu', 'layer1[0].conv2', 'layer1[0].bn2', 'layer1[1].conv1', 'layer1[1].bn1', 'layer1[1].relu', 'layer1[1].conv2', 'layer1[1].bn2', 'layer2[0].conv1', 'layer2[0].bn1', 'layer2[0].relu', 'layer2[0].conv2', 'layer2[0].bn2', 'layer2[0].downsample[0]', 'layer2[0].downsample[1]', 'layer2[1].conv1', 'layer2[1].bn1', 'layer2[1].relu', 'layer2[1].conv2', 'layer2[1].bn2', 'layer3[0].conv1', 'layer3[0].bn1', 'layer3[0].relu', 'layer3[0].conv2', 'layer3[0].bn2', 'layer3[0].downsample[0]', 'layer3[0].downsample[1]', 'layer3[1].conv1', 'layer3[1].bn1', 'layer3[1].relu', 'layer3[1].conv2', 'layer3[1].bn2', 'layer4[0].conv1', 'layer4[0].bn1', 'layer4[0].relu', 'layer4[0].conv2', 'layer4[0].bn2', 'layer4[0].downsample[0]', 'layer4[0].downsample[1]', 'layer4[1].conv1', 'layer4[1].bn1', 'layer4[1].relu', 'layer4[1].conv2', 'layer4[1].bn2', 'avgpool', 'fc']


In [15]:
# output needs to be evaluated after adding the layer_hook
inp = torch.ones(1,3,32,32, requires_grad=True)
outp = model(inp)   

print(model.state_dict().keys())

write_layer_outputs_to_file         = True
write_fused_layer_params_to_file    = True
write_layer_params_to_file          = False

EPS = 10 ** -5 # constant

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.num_batches_tracked', 'layer2.0.conv1.weight', 'layer2.0.bn1.weight', 'layer2.0.bn1.bias', 'layer2.0.bn1.running_mean', 'layer2.0.bn1.running_var', 'layer2.0.bn1.num_batches_tracked', 'layer2.0.conv2.weight', 'layer2.0.bn2.weight', 'layer2.0.bn2.bias', '

In [16]:

if(write_layer_outputs_to_file):
    # Write layer outputs
    for i in range(len(layers)):
        layer = layers[i]
        print("Outputs ",layer)
        if(layer in features.keys()):
            layer_name = layer.replace("].","_")
            layer_name = layer_name.replace("[", "_")
            layer_name = layer_name.replace("]", "")
            filename = "bin/" +  layer_name + ".bin"
            with open(filename,"wb") as f:
                features[layer].cpu().numpy().tofile(f)
            print("Layer " + str(i) + " feature map printed to " + filename)
            print("Shape:", features[layer].cpu().numpy().shape)

Outputs  conv1
Layer 0 feature map printed to bin/conv1.bin
Shape: (1, 64, 16, 16)
Outputs  bn1
Layer 1 feature map printed to bin/bn1.bin
Shape: (1, 64, 16, 16)
Outputs  relu
Layer 2 feature map printed to bin/relu.bin
Shape: (1, 64, 16, 16)
Outputs  maxpool
Layer 3 feature map printed to bin/maxpool.bin
Shape: (1, 64, 8, 8)
Outputs  layer1[0].conv1
Layer 4 feature map printed to bin/layer1_0_conv1.bin
Shape: (1, 64, 8, 8)
Outputs  layer1[0].bn1
Layer 5 feature map printed to bin/layer1_0_bn1.bin
Shape: (1, 64, 8, 8)
Outputs  layer1[0].relu
Layer 6 feature map printed to bin/layer1_0_relu.bin
Shape: (1, 64, 8, 8)
Outputs  layer1[0].conv2
Layer 7 feature map printed to bin/layer1_0_conv2.bin
Shape: (1, 64, 8, 8)
Outputs  layer1[0].bn2
Layer 8 feature map printed to bin/layer1_0_bn2.bin
Shape: (1, 64, 8, 8)
Outputs  layer1[1].conv1
Layer 9 feature map printed to bin/layer1_1_conv1.bin
Shape: (1, 64, 8, 8)
Outputs  layer1[1].bn1
Layer 10 feature map printed to bin/layer1_1_bn1.bin
Shape:

In [12]:
if(write_layer_params_to_file):
    # Write layer params
    for i in range(len(layers)):
        layer = layers[i]
        print(layer)
        if('conv' in layer or 'downsample[0]' in layer):
            layer_name = layer.replace("].","_")
            layer_name = layer_name.replace("[", "_")
            layer_name = layer_name.replace("]", "")
            filename = "bin/" +  layer_name + "_weights.bin"
            param_name = layer.replace("[",".")
            param_name = param_name.replace("]","")
            param = model.state_dict()[param_name+'.weight'].detach().numpy()
            with open(filename, "wb") as f:
                param.tofile(f)
            print("Param " + param_name + " printed to file " + filename)

        if('bn' in layer or 'downsample[1]' in layer):
            layer_name = layer.replace("].","_")
            layer_name = layer_name.replace("[", "_")
            layer_name = layer_name.replace("]", "")
            filename = "bin/" +  layer_name + "_params.bin"
      
            param_name = layer.replace("[",".")
            param_name = param_name.replace("]","")
      
            weight      = model.state_dict()[param_name+'.weight']
            sqrt_var    = torch.sqrt(model.state_dict()[param_name+'.running_var'] + EPS)
            wt_sqrt_var = weight / sqrt_var
      
            param = numpy.concatenate([
                wt_sqrt_var.detach().numpy(),
                model.state_dict()[param_name+'.bias'].detach().numpy(),
                model.state_dict()[param_name+'.running_mean'].detach().numpy()])

            with open(filename, "wb") as f:
                param.tofile(f)
            print("Param " + param_name + " printed to file " + filename)

In [14]:
if(write_fused_layer_params_to_file):
    # Write layer params
    for i in range(len(layers)):
        layer = layers[i]
        print(layer)
        if('linear' in layer):
            linear_weight = model.state_dict()[layer+'.weight'].detach().numpy()
            filename = "bin/" + layer + "_weights.bin"
            with open(filename, "wb") as f:
                linear_weight.tofile(f)
            print("Param " + layer + "weights"+ " printed to file " + filename)
            print("Shape:", linear_weight.shape)
            
            linear_bias = model.state_dict()[layer+'.bias'].detach().numpy()
            filename = "bin/" + layer + "_bias.bin"
            with open(filename, "wb") as f:
                linear_bias.tofile(f)
            print("Param " + layer + "bias"+ " printed to file " + filename)
            print("Shape:", linear_bias.shape)
            
        
        if('conv' in layer or 'downsample[0]' in layer or 'shortcut[0]' in layer):
            conv_layer_name = layer.replace("].","_")
            conv_layer_name = conv_layer_name.replace("[", "_")
            conv_layer_name = conv_layer_name.replace("]", "")

            conv_param_name = layer.replace("[",".")
            conv_param_name = conv_param_name.replace("]","")

            conv_weight = model.state_dict()[conv_param_name+'.weight']

        if('bn' in layer or 'downsample[1]' in layer or 'shortcut[1]' in layer):
            bn_layer_name = layer.replace("].","_")
            bn_layer_name = bn_layer_name.replace("[", "_")
            bn_layer_name = bn_layer_name.replace("]", "")

            bn_param_name = layer.replace("[",".")
            bn_param_name = bn_param_name.replace("]","")

            bn_weight = model.state_dict()[bn_param_name+'.weight']
            bn_bias   = model.state_dict()[bn_param_name+'.bias']
            bn_mean   = model.state_dict()[bn_param_name+'.running_mean']
            bn_var    = model.state_dict()[bn_param_name+'.running_var']

            bn_factor    = torch.div(bn_weight,torch.sqrt(bn_var+EPS)).view(-1,1,1,1)
            fused_weight = torch.mul(conv_weight, bn_factor)
            fused_bias   = bn_bias - torch.div(torch.mul(bn_weight,bn_mean),torch.sqrt(bn_var+EPS))
            
            if('downsample' in bn_layer_name):
                layer_number = '0'
                layer_prefix = bn_layer_name[0:bn_layer_name.find('downsample')]
            else:
                layer_number = conv_layer_name[-1]
                layer_prefix = bn_layer_name[0:bn_layer_name.find('bn')]
        
            weights_filename = "bin/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_weights.bin"
            bias_filename    = "bin/fused_" + layer_prefix + "conv" + layer_number + "_bn" + layer_number + "_bias.bin"
            with open(weights_filename, "wb") as f:
                fused_weight.detach().numpy().tofile(f)
            print("Fused weights of " + layer_prefix + layer_number + " printed to file " + weights_filename)
            print("Shape:", fused_weight.shape)
        
            with open(bias_filename, "wb") as f:
                fused_bias.detach().numpy().tofile(f)
            print("Fused biases of " + layer_prefix + layer_number + " printed to file " + bias_filename)
            print("Shape:", fused_bias.shape)

conv1
bn1
Fused weights of 1 printed to file bin/fused_conv1_bn1_weights.bin
Shape: torch.Size([64, 3, 7, 7])
Fused biases of 1 printed to file bin/fused_conv1_bn1_bias.bin
Shape: torch.Size([64])
relu
maxpool
layer1[0].conv1
layer1[0].bn1
Fused weights of layer1_0_1 printed to file bin/fused_layer1_0_conv1_bn1_weights.bin
Shape: torch.Size([64, 64, 3, 3])
Fused biases of layer1_0_1 printed to file bin/fused_layer1_0_conv1_bn1_bias.bin
Shape: torch.Size([64])
layer1[0].relu
layer1[0].conv2
layer1[0].bn2
Fused weights of layer1_0_2 printed to file bin/fused_layer1_0_conv2_bn2_weights.bin
Shape: torch.Size([64, 64, 3, 3])
Fused biases of layer1_0_2 printed to file bin/fused_layer1_0_conv2_bn2_bias.bin
Shape: torch.Size([64])
layer1[1].conv1
layer1[1].bn1
Fused weights of layer1_1_1 printed to file bin/fused_layer1_1_conv1_bn1_weights.bin
Shape: torch.Size([64, 64, 3, 3])
Fused biases of layer1_1_1 printed to file bin/fused_layer1_1_conv1_bn1_bias.bin
Shape: torch.Size([64])
layer1[1].rel

In [ ]:
inp=torch.ones(1,3,96,96,requires_grad=True,device=device)

filename="bin/inputs/input.bin"

with open(filename,"wb") as f:

inp.detach().numpy().tofile(f)